In [ ]:
import os
import pickle
import sys
import math
sys.path.append("../")
from od.env.config import SUMO_SIM_SECONDS
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,ROOT_DIR
from od.env.station import BS_PRESET
import od.misc.logger
import od.misc.statistic as oms
from datetime import datetime

In [ ]:
class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial

    


In [ ]:
def Run(statistic,interest_config):
    statistic.nft_traffic = {}
    statistic.raw_sg_header = None
    statistic.interest_config = interest_config
    statistic.dirpath = "../data/GS20/" + statistic.interest_config.folder()
    print(statistic.dirpath)
    BS_CONFIG =  {
            name:bs_config 
            for name,bs_config in BS_PRESET.items()
            if (
                bs_config["type"] == BaseStationType.UMA  or 
                (
                    interest_config.req_rsu and 
                    bs_config["type"] == BaseStationType.UMI
                )
            )
    }  
    time_text = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    class GV:
        NET_STATION_CONTROLLER = [ 
            FakeBaseStation(
                name,
                BS_CONFIG[name]["pos"],
                BS_CONFIG[name]["type"] ,
                i
            )   for i,name in enumerate(BS_CONFIG)
        ]
        STATISTIC = od.misc.logger.Logger(statistic.dirpath,"Statistic ({}).txt".format(time_text))
        RESULT = od.misc.logger.Logger(statistic.dirpath,"Result ({}).txt".format(time_text))
    oms.GV = GV
    statistic.Report(True)
    GV.STATISTIC.Encapsulate()
    GV.RESULT.Encapsulate()
    

In [ ]:
# interest_config = InterestConfig(False,ResourceAllocatorType.NOMA_OPT,False,1.4,9)
# old_statistic = None
# with open("../data/{}/object.pickle".format(interest_config.folder_legacy()), "rb") as file:
#     old_statistic = pickle.load(file)
# Run(old_statistic,interest_config)

In [ ]:
for qos_re_class in [False]:
        for res_alloc_type in [ResourceAllocatorType.NOMA_OPT]:
            for rsu in [False, True]:
                for traffic_scale in [i/10 for i in range(7, 15, 1)]:
                    for seed in [ i+ 1 for i in range(20)]:
                        interest_config = InterestConfig(qos_re_class,res_alloc_type,rsu,traffic_scale,seed)
                        old_statistic = None
                        with open("../data/{}/object.pickle".format(interest_config.folder_legacy()), "rb") as file:
                            old_statistic = pickle.load(file)
                        Run(old_statistic,interest_config)